# Amazon Bedrock Fundamental

이 노트북은, SageMaker Notebook conda_python3 커널에서 테스트 되었습니다.

## 0. 환경 설정

In [ ]:
# 필요한 라이브러리들을 이 환경에 설치합니다.

!pip install -r ./requirements.txt

# 1. Bedrock Embedding 호출

In [ ]:
import json
import boto3
from numpy import dot
from numpy.linalg import norm

# Bedrock 임베딩 가져오는 함수
def get_embedding(text):
    session = boto3.Session()
    bedrock = session.client(service_name='bedrock-runtime')

    response = bedrock.invoke_model(
        body=json.dumps({
            "inputText": text,
            "dimensions": 1024
        }),
        modelId="amazon.titan-embed-text-v2:0", # Titan Text Embeddings V2
        accept="application/json",
        contentType="application/json"
    )

    response_body = json.loads(response['body'].read())
    return response_body['embedding']


# 코사인 유사도 계산 함수
def calculate_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

In [ ]:
# 테스트 문장
test_sentence = "인공지능은 현대 기술의 핵심 분야입니다."
test_embedding = get_embedding(test_sentence)

compare_sentences = [
    "축구는 전 세계에서 가장 인기 있는 스포츠 중 하나입니다.",
    "클래식 음악은 많은 사람들에게 평화를 줍니다.",
    "AI는 오늘날 기술 혁신의 중심에 있습니다.",
    "현대 기술에서 인공지능의 역할은 매우 중요합니다.",
    "고대 이집트의 피라미드는 건축학적 경이로움입니다.",
    "머신러닝은 현대 기술 발전의 주요 동력입니다.",
    "지구 온난화는 현대 사회의 심각한 환경 문제입니다.",
    "건강한 식단은 장수의 비결 중 하나입니다.",
    "바다에는 아직 발견되지 않은 많은 생물 종이 있습니다."
]

compare_results = []
for compare_sentence in compare_sentences:
    compare_embedding = get_embedding(compare_sentence)
    similarity = calculate_similarity(test_embedding, compare_embedding)
    compare_results.append((compare_sentence, similarity))

# 유사도를 기준으로 내림차순 정렬
compare_results.sort(key=lambda x: x[1], reverse=True)

print("# 테스트 문장")
print(f"\"{test_sentence}\"")

print("\n# 임베딩 비교 결과 (유사도 높은 순)")
for i, (sentence, similarity) in enumerate(compare_results, 1):
    print(f"{i}. 유사도: {similarity:.4f} - \"{sentence}\"")

# 2. Bedrock Claude 3 호출

## 2-1. 텍스트 생성

In [ ]:
import boto3
import json

# Bedrock 클라이언트 생성
bedrock_runtime = boto3.client('bedrock-runtime')

# Claude 3 Haiku 모델 ID
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# 시스템 프롬프트 설정
system = [{"text": "당신은 친절한 AI 친구입니다. 상냥하게 답변해주세요."}]

# 대화 메시지 설정
messages = [
    {"role": "user", "content": [{"text": "안녕하세요, 당신은 누구인가요?"}]}
]

# 하이퍼파라미터 설정
inference_config={"maxTokens": 512, "temperature": 0.5, "topP": 0.9}
 
# converse API 호출
try:
    response = bedrock_runtime.converse(
        modelId=model_id,
        messages=messages,
        system=system,
        inferenceConfig=inference_config
    )
    
    # 응답 처리
    ai_message = response["output"]["message"]
    output_text = ai_message["content"][0]["text"]
    print("# Claude 3 Haiku의 응답")
    print(f"- {output_text}")
    print()
        
    
    # 토큰 사용량 출력
    token_usage = response["usage"]
    print("# 토큰 사용량")
    print(f"- 입력 토큰: {token_usage['inputTokens']}")
    print(f"- 출력 토큰: {token_usage['outputTokens']}")
    print(f"- 총 토큰: {token_usage['totalTokens']}")

except Exception as e:
    print(f"오류 발생: {str(e)}")

## 2-2. 텍스트 생성 - 스트리밍

In [ ]:
import boto3
import json

# Bedrock 클라이언트 생성
bedrock_runtime = boto3.client('bedrock-runtime')

# Claude 3 Haiku 모델 ID
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# 시스템 프롬프트 설정
system = [{"text": "당신은 친절한 AI 친구입니다. 상냥하게 답변해주세요."}]

# 대화 메시지 설정
messages = [
    {"role": "user", "content": [{"text": "안녕하세요, 당신은 누구인가요?"}]}
]

# 하이퍼파라미터 설정
inference_config={"maxTokens": 512, "temperature": 0.5, "topP": 0.9}
 
# converse API 호출
try:
    response = bedrock_runtime.converse_stream(
        modelId=model_id,
        messages=messages,
        system=system,
        inferenceConfig=inference_config
    )
    
    # 스트리밍 응답 처리
    print("# AI 응답\n-", end=" ", flush=True)
    for event in response['stream']:
        if 'contentBlockDelta' in event:
            content_delta = event['contentBlockDelta']
            if 'delta' in content_delta and 'text' in content_delta['delta']:
                print(content_delta['delta']['text'], end="", flush=True)
        elif 'messageStop' in event:
            print()
            pass
        elif 'metadata' in event:
            metadata = event['metadata']
            if 'usage' in metadata:
                # 토큰 사용량 출력
                print("\n# 토큰 사용량")
                print(f"- 입력 토큰: {metadata['usage']['inputTokens']}")
                print(f"- 출력 토큰: {metadata['usage']['outputTokens']}")
                print(f"- 총 토큰: {metadata['usage']['totalTokens']}")
                print()
            if 'metrics' in event['metadata']:
                print(
                    f"# Latency\n- {metadata['metrics']['latencyMs']} milliseconds")


except Exception as e:
    print(f"오류 발생: {str(e)}")

## 2-3. 이미지 분석

In [ ]:
import os
import boto3
import base64
import json
from IPython.display import Image, display

# Bedrock 클라이언트 생성
bedrock_runtime = boto3.client('bedrock-runtime')

# Claude 3 Haiku 모델 ID
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# 이미지 파일 경로
image_path = "image/sample_image_camping.jpeg"

# 이미지 표시
display(Image(filename=image_path))

In [ ]:
# 이미지를 리드
with open(image_path, "rb") as image_file:
    encoded_image = image_file.read()

# 대화 메시지 설정
messages = [
    {
        "role": "user",
        "content": [
            {
                "image": {
                    "format": "jpeg",
                    "source": {
                        "bytes": encoded_image
                    }
                }
            },
            {"text": "이 이미지에 대해 자세히 설명해주세요."}
        ]
    }
]


# 하이퍼파라미터 설정
inference_config = {"temperature": 0.5, "topP": 0.9}

# converse API 호출
try:
    response = bedrock_runtime.converse(
        modelId=model_id,
        messages=messages,
        inferenceConfig=inference_config,
    )
    
    # 응답 처리
    ai_message = response["output"]["message"]
    output_text = ai_message["content"][0]["text"]
    print("Claude 3 Haiku의 이미지 분석 결과:")
    print(output_text)
    
    # 토큰 사용량 출력
    token_usage = response["usage"]
    print(f"\n입력 토큰: {token_usage['inputTokens']}")
    print(f"출력 토큰: {token_usage['outputTokens']}")
    print(f"총 토큰: {token_usage['totalTokens']}")

except Exception as e:
    print(f"오류 발생: {str(e)}")